In [1]:
import pandas as pd
data=[]
code=[]
for i in range(1,38):
    sheet_name='Sheet0 ('+str(i)+')'
    print(sheet_name)
    temp_data=pd.read_excel('附件1.xlsx',sheet_name=sheet_name,converters={'证券代码':str})
    data.append(temp_data)
    code.append(temp_data.iloc[0,0])

Sheet0 (1)
Sheet0 (2)
Sheet0 (3)
Sheet0 (4)
Sheet0 (5)
Sheet0 (6)
Sheet0 (7)
Sheet0 (8)
Sheet0 (9)
Sheet0 (10)
Sheet0 (11)
Sheet0 (12)
Sheet0 (13)
Sheet0 (14)
Sheet0 (15)
Sheet0 (16)
Sheet0 (17)
Sheet0 (18)
Sheet0 (19)
Sheet0 (20)
Sheet0 (21)
Sheet0 (22)
Sheet0 (23)
Sheet0 (24)
Sheet0 (25)
Sheet0 (26)
Sheet0 (27)
Sheet0 (28)
Sheet0 (29)
Sheet0 (30)
Sheet0 (31)
Sheet0 (32)
Sheet0 (33)
Sheet0 (34)
Sheet0 (35)
Sheet0 (36)
Sheet0 (37)


In [18]:
all_list=pd.read_csv('股票列表.csv',converters={'symbol':str})
list_code=[]
list_date=[]
for i in code:
    temp_list=all_list[all_list['symbol']==i]
    list_code.append(temp_list.iloc[0,0])
    list_date.append(temp_list.iloc[0,5])

In [28]:
import tushare as ts
pro = ts.pro_api()
for i in range(len(list_code)):
    df = pro.daily_basic(ts_code=str(list_code[i]), start_date=str(list_date[i]), end_date='20210528',fields='ts_code,trade_date,close,total_share,float_share,total_mv,circ_mv')
    df.to_csv('每日股本/'+str(list_code[i][0:6])+'.csv',index=False)

In [37]:
import os
files=os.listdir('每日股本')
all_list=pd.read_csv('每日股本/'+(files[0]))
for i in range(1,len(files)):
    temp_data=pd.read_csv('每日股本/'+files[i])
    all_list=pd.concat([all_list,temp_data])
all_list.to_csv('每日股本总数据.csv',index=False)

In [46]:
base=all_list[all_list['trade_date']==20190102]
base=sum(base['total_mv'].tolist())
base

25564188.9392

In [54]:
date_data=pd.read_csv('每日股本/000012.csv')
date_data=date_data[date_data['trade_date']>20190102]['trade_date'].tolist()
date_data.sort()
def index_count(temp_date,temp_base,data):
    temp_date=temp_date
    temp_base=temp_base
    data=data
    index=data[data['trade_date']==temp_date]
    index=sum(index['total_mv'].tolist())
    index=index/temp_base*100
    return index

103.49983136890395

In [64]:
#计算调整节点
first_change=date_data[date_data.index(20190123)+14]
first_change_index=date_data.index(20190123)+14
second_change=date_data[date_data.index(20210129)+14]
second_change_index=date_data.index(20210129)+14

In [82]:
stock_index=[]
first_list=all_list[all_list['ts_code']!='601615.SH']
for i in range(0,first_change_index):
    date=date_data[i]
    temp_index=index_count(date,base,first_list)
    stock_index.append([date,temp_index])

In [93]:
#计算新的基期总值
index=all_list[all_list['trade_date']==20190219]
index=sum(index['total_mv'].tolist())
change=all_list[(all_list['ts_code']=='601615.SH')&(all_list['trade_date']==20190219)]
change=change.iloc[0,5]
new_base=(index)*base/(index-change)

27093481.21372206

In [96]:
first_list=all_list[all_list['ts_code']!='003035.SZ']
for i in range(first_change_index,second_change_index):
    date=date_data[i]
    temp_index=index_count(date,new_base,first_list)
    stock_index.append([date,temp_index])

In [99]:
index=all_list[all_list['trade_date']==20210225]
index=sum(index['total_mv'].tolist())
change=all_list[(all_list['ts_code']=='003035.SZ')&(all_list['trade_date']==20210225)]
change=change.iloc[0,5]
final_base=(index)*new_base/(index-change)

28014487.543591984

In [100]:
first_list=all_list
for i in range(second_change_index,len(date_data)):
    date=date_data[i]
    temp_index=index_count(date,new_base,first_list)
    stock_index.append([date,temp_index])

In [102]:
stock_index=pd.DataFrame(stock_index)

,0,1
0,20190103,99.137837
1,20190104,101.729362
2,20190107,103.499831
3,20190108,103.996457
4,20190109,104.616625
...,...,...
576,20210521,269.828599
577,20210524,265.772238
578,20210525,267.575215
579,20210526,266.821382


In [104]:
stock_index.to_csv('stock_index.csv',index=False)
stock_index.to_excel('stock_index.xlsx',index=False)